In [1]:
import os

import pandas as pd
from bs4 import BeautifulSoup
from tqdm import tqdm

In [2]:
path_artigos = os.path.join(os.getcwd(), 'artigos_brutos')

In [3]:
# contagem de artigos
soma = len(os.listdir(path_artigos))
print(f'Total de artigos: {soma}')

Total de artigos: 1681


# Criando o Dataset

In [4]:
dict_df = {'title': [], 'tipo': [], 'ano': [],  'doi_id': [], 'file_bruto_name': []}

for idx, arquivo in tqdm(enumerate(os.listdir(path_artigos), start=1)):

    path_abs  = os.path.join(path_artigos, arquivo)

    with open(path_abs, 'r', encoding='utf8') as f:
        texto_html = f.read()

    soup = BeautifulSoup(texto_html, 'lxml')

    try:
        doi_id = soup.find('a', attrs={'class': '_doi'}).text.strip().removeprefix('https://doi.org/')
    except AttributeError:
        doi_id = ''

    try:
        titulo = ' '.join(soup.find('h1', attrs={'class': 'article-title'}).text.split())
    except AttributeError:
        titulo = ''

    try:
        tipo = soup.find('span', attrs={'class': '_articleBadge'}).text.strip().lower()
    except AttributeError:
        tipo = ''

    try:
        ano = soup.find('span', attrs={'class': '_editionMeta'}).text.split('•')[-1].strip()
        if len(ano) > 4:
            ano = ano[-4:]
    except AttributeError:
        ano = ''

    dict_df['title'].append(titulo)
    dict_df['doi_id'].append(doi_id)
    dict_df['file_bruto_name'].append(arquivo)
    dict_df['tipo'].append(tipo)
    dict_df['ano'].append(int(ano))


1681it [02:12, 12.73it/s]


In [5]:
df = pd.DataFrame(dict_df)

In [6]:
df.head()

,title,tipo,ano,doi_id,file_bruto_name
0,Mestrado Nacional Profissional em Ensino de Fí...,cartas ao editor,2022,10.1590/1806-9126-RBEF-2021-0392,0000.html
1,A equivalência entre o Princípio de Maximizaçã...,cartas ao editor,2022,10.1590/1806-9126-RBEF-2021-0407,0001.html
2,O impacto do MNPEF no desenvolvimento científi...,cartas ao editor,2022,10.1590/1806-9126-RBEF-2022-0128,0002.html
3,Quebra espontânea de simetria e mecanismo de H...,artigos gerais,2022,10.1590/1806-9126-RBEF-2021-0264,0003.html
4,Sombras de buracos negros: desvendando a físic...,artigos gerais,2021,10.1590/1806-9126-RBEF-2020-0232,0004.html


In [7]:
df.to_csv('scielo_dataset.csv', index=False)  # salva o arquivo